In [5]:
import pandas as pd
import numpy as np
import re
from transformers import pipeline
file_path = "C:\\Users\\zhi-n\\Documents\\FIT3163\\mimic-iv-note\\discharge.csv"
data = pd.read_csv(file_path)

c:\Users\zhi-n\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
data.head(5)
summarizer = pipeline("summarization", model="Falconsai/medical_summarization", tokenizer="t5-large")

In [18]:
def extract_sections(note):
    """Extracts sections from the medical note based on headers."""
    sections = {}
    lines = note.split('\n')
    current_header = None
    content = []
    for line in lines:
        line = line.strip()
        if re.match(r'^[A-Z][\w\s-]*(?::)\s*$', line):
            if current_header is not None:
                sections[current_header] = ' '.join(content)
            current_header = line.rstrip(':')
            content = []
        else:
            content.append(line)
    if current_header is not None:
        sections[current_header] = ' '.join(content)
    return sections

def clean_text(text):
    """Cleans the text by removing unnecessary characters and spaces."""
    text = re.sub(r'_{2,}|={2,}|-{2,}', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def summarize_section(header, text):
    """Summarizes the text using a pre-trained summarization model and includes the header."""
    cleaned_text = clean_text(text)
    tokens = cleaned_text.split()
    token_length = len(tokens)

    if token_length < 30:  # Consider skipping very short texts
        return f"{header}: {cleaned_text}"  # Include the header in the output

    try:
        max_len = min(max(150, int(token_length * 0.75)), token_length)  # Dynamically setting max_length
        summary = summarizer(
            cleaned_text,
            max_length=max_len,
            min_length=10,  # Minimum length of summary
            truncation=True
        )[0]['summary_text']
        return f"{header}: {summary}"  # Prefix the summary with its section header
    except Exception as e:
        print(f"Error during summarization: {str(e)}")
        return f"{header}: {cleaned_text}"  # Fallback to using cleaned text if an error occurs


# Get the text from the first row
first_row_text = data.loc[0, 'text']

# Extract and summarize sections from the first row's text
sections = extract_sections(first_row_text)
summarized_sections = [summarize_section(header, content) for header, content in sections.items()]

# Combine all summaries into a single string
full_summary = ' '.join(summarized_sections)

# Print the full summary
print(full_summary)

Allergies: No Known Allergies / Adverse Drug Reactions Attending: Chief Complaint: Worsening ABD distension and pain Major Surgical or Invasive Procedure: Paracentesis History of Present Illness: we report a case of a patient with cirrhosis c/b ascites , hiv on ART , h/o IVDU, COPD, bioplar, PTSD , presented from OSH ED with worsening abd distension over past week . pt reports self-discontinuing lasix and spirnolactone weeks ago , because she feels like "they don't do anything" and that she "doesn't want put more chemicals in her . Past Medical History: a 15 mm hypoechoic lesion on her ultrasound on and is being monitored by an MRI . she also had another lesion in her forehead with purple discoloration . it was biopsied to exclude the possibility of 's sarcoma, the results is pending . Social History:  Family History: she a total of five siblings , but she is not talking to most of them . she only has one brother that she is in touch with and lives in . her last alcohol consumption was